# **Incremental Parameter**

In [30]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 5143f611-bf85-4507-a91e-da144ce3e206, 35, Finished, Available, Finished)

# **CREATING DIMENSION -branch**

In [31]:
df_src=spark.sql('''
SELECT DISTINCT(Branch_ID),BranchName
FROM parquet.`abfss://incre_ws@onelake.dfs.fabric.microsoft.com/lake_san.Lakehouse/Files/silver/carsales`
''')

StatementMeta(, 5143f611-bf85-4507-a91e-da144ce3e206, 36, Finished, Available, Finished)

In [32]:
display(df_src)

StatementMeta(, 5143f611-bf85-4507-a91e-da144ce3e206, 37, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, eaa4f41e-f327-4460-86ed-3950c80f47e0)

## initial and incremental

In [33]:
if spark.catalog.tableExists("dim_branch"):
    df_sink=spark.sql('''
    SELECT dim_Branch_key,Branch_ID,BranchName
    FROM dim_branch''')
else:
    df_sink=spark.sql('''
    SELECT 1 AS dim_Branch_key,Branch_ID,BranchName
    FROM parquet.`abfss://incre_ws@onelake.dfs.fabric.microsoft.com/lake_san.Lakehouse/Files/silver/carsales`
    WHERE 1=0 ''') #creating empty table with schema

StatementMeta(, 5143f611-bf85-4507-a91e-da144ce3e206, 38, Finished, Available, Finished)

In [34]:
df_filter=df_src.join(df_sink,df_src['Branch_ID']==df_sink['Branch_ID'],'left').select(df_src['Branch_ID'],df_src['BranchName'],df_sink['dim_Branch_key'])
df_old=df_filter.filter(col('dim_Branch_key').isNotNull())
df_new=df_filter.filter(col('dim_Branch_key').isNull()).select(df_src['Branch_ID'],df_src['BranchName'])

StatementMeta(, 5143f611-bf85-4507-a91e-da144ce3e206, 39, Finished, Available, Finished)

In [35]:
display(df_new)

StatementMeta(, 5143f611-bf85-4507-a91e-da144ce3e206, 40, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 431ed1ab-825e-4aaa-8333-285387be9390)

## Surrogate key creation

In [36]:
if spark.catalog.tableExists("dim_branch"):
    max_value_df=spark.sql('''SELECT MAX(dim_Branch_key) FROM dim_branch''') #return as a dataset
    max_value=max_value_df.collect()[0][0]
else:
    max_value=1

StatementMeta(, 5143f611-bf85-4507-a91e-da144ce3e206, 41, Finished, Available, Finished)

In [37]:
df_new = df_new.withColumn(
    "dim_Branch_key",
    monotonically_increasing_id() + 1)


StatementMeta(, 5143f611-bf85-4507-a91e-da144ce3e206, 42, Finished, Available, Finished)

In [38]:
display(df_new)

StatementMeta(, 5143f611-bf85-4507-a91e-da144ce3e206, 43, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1de6efea-04c8-40f3-965f-e63ce67c69cf)

In [39]:
df_final=df_new.union(df_old)

StatementMeta(, 5143f611-bf85-4507-a91e-da144ce3e206, 44, Finished, Available, Finished)

# **SCD TYPE-1: UPSERT**

In [40]:
from delta.tables import DeltaTable

StatementMeta(, 5143f611-bf85-4507-a91e-da144ce3e206, 45, Finished, Available, Finished)

In [41]:
if spark.catalog.tableExists('dim_branch'):            #Incremental run
    delta_table=DeltaTable.forPath(spark,'abfss://incre_ws@onelake.dfs.fabric.microsoft.com/lake_san.Lakehouse/Files/gold/dim_branch')
    delta_table.alias("trg").merge(df_final.alias("src"),"trg.Branch_ID=src.Branch_ID")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()
else:                                                      #Initial run
    df_final.write.format("delta")\
        .mode('overwrite')\
        .option('path','abfss://incre_ws@onelake.dfs.fabric.microsoft.com/lake_san.Lakehouse/Files/gold/dim_branch')\
        .saveAsTable('dim_branch')

StatementMeta(, 5143f611-bf85-4507-a91e-da144ce3e206, 46, Finished, Available, Finished)

In [42]:
%%sql
SELECT * FROM dim_branch

StatementMeta(, 5143f611-bf85-4507-a91e-da144ce3e206, 47, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 3 fields>

In [1]:
%%sql
SELECT COUNT(*) FROM dim_branch

StatementMeta(, 7042aaa6-e370-4d50-8de6-7c9412deb482, 2, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>